# PROYECTO OPEN AI RAG 

In [ ]:
#pip install langchain langchain_openai gradio chromadb pypdf

In [ ]:
# pip install -U langchain-community

In [9]:
pdf1 = r"https://www.boe.es/boe/dias/2022/09/29/pdfs/BOE-A-2022-15818.pdf"
pdf2 = r"https://www.camara.es/sites/default/files/publicaciones/fiscalidad03.pdf"
pdf3 = r"https://mpt.gob.es/dam/es/portal/delegaciones_gobierno/delegaciones/madrid/proyectos-ci/Guia-de-Subvenciones-y-Ayudas/231103_Guia_de_Subvenciones_Ayudas_PRTR.pdf.pdf"
pdf4 = r"https://boe.es/buscar/pdf/2014/BOE-A-2014-12328-consolidado.pdf"
pdf5 = r"https://www.aepd.es/guias/gestion-riesgo-y-evaluacion-impacto-en-tratamientos-datos-personales.pdf"
pdf6 = r"https://ipyme.org/PUBLICACIONES_EMPRESAS/Ciclo%20Vital%20de%20la%20Empresa/CreacionEmpresas.pdf"

#### LIBRERIAS NECESARIAS

In [8]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma 
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import gradio as gr

In [10]:
urls = [pdf1, pdf2, pdf3, pdf4, pdf5, pdf6] #lista de urls de los pdfs

#### PREPOCESAMIENTO DE LA DOCUMENTACIÓN

In [11]:
pages = []
for url in urls:
    loader = PyPDFLoader(url) #loader hace el download de los pdfs
    pages += loader.load_and_split() #con pages los uno para que esten todos en un doc. split hace que sigan teniendo paginas separadas

In [12]:
len(pages) #verifico el numero de paginas para ver que se ha unido correctamente

931

In [ ]:
pages[54] #acceder a pagina 54 por ejemplo

#### CONFIGURACION DE LLM

In [15]:
#creamos el llm para crear el modelo, aqui cogemos la info de la documentacion de platform open ai. Vamos a coger el modelo gpt3.5 con nuestra api key, le ponemos temperatura para definir el tono de respuesta, que no sea muy creativo
llm = ChatOpenAI(model='gpt-3.5-turbo-0125', 
                 api_key='', 
                 temperature= 0)

In [18]:
#ahora tenemos que hacer que la documentacion de pages aparezca como un vector para que sea legible. Lo que se llama embedding. Usamos directamente OpenAIEmbeddings
embedding = OpenAIEmbeddings(api_key='')

#### CREAR UN ALMACEN PARA ESOS VECTORES (DB)

In [20]:
# para crear la DB usamos Chroma
vectorstore = Chroma.from_documents(
    documents = pages,
    embedding = embedding, 
    )

In [21]:
retriever = vectorstore.as_retriever() # nos permite recuperar documentos similares a un texto de entrada

#### PROMPTING DE COMPORTAMIENTO
Vamos a definir como entramos en esa DB

In [22]:
template = """ 

        ###INSTRUCCIONES: 
        Eres un asistente IA del gobierno de España dedicado a responder preguntas empresariales de manera educada y profesional. Debes proporcionar una respuesta útil al usuario. 
        
        En tu respuesta, POR FAVOR SIEMPRE:
          (0) Sé un lector atento a los detalles: lee la pregunta y el contexto y entiende ambos antes de responder.
          (1) Comienza tu respuesta con un tono amigable y reitera la pregunta para que el usuario esté seguro de que la entendiste.
          (2) Si el contexto te permite responder a la pregunta, escribe una respuesta detallada y técnica, útil y fácil de entender, con fuentes referenciadas en el texto. SI NO: no puedes encontrar la respuesta, responde con una explicación, comenzando con: "No pude encontrar la información en las leyes y documentos a los que tengo acceso".
          (3) Debajo de la respuesta, por favor enumera todas las fuentes referenciadas (es decir, párrafos legales que respaldan tus afirmaciones).
          (4) Ahora que tienes tu respuesta, fantástico - revisa tu respuesta para asegurarte de que respondes a la pregunta, esta respuesta es útil y profesional , formateada para ser fácilmente legible.
        
        
        PIENSA PASO POR PASO 
        ###
        
      Responde a la siguiente pregunta utilizando el contexto proporcionado.
        ### Question: {question} ###
        ### Context: {context} ###     
        ### Respuesta Útil con Fuentes:

"""

In [23]:
prompt = PromptTemplate.from_template(template)

#### CADENA O CHAIN


In [28]:
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### PREGUNTA DE CONTROL

In [29]:
ans = chain.invoke('Que documentacion necesito para montar una SL en España')
print(ans)

¡Hola! Para montar una Sociedad de Responsabilidad Limitada (SL) en España, necesitarás la siguiente documentación:

1. Documento de identidad de los socios.
2. Certificación negativa de la denominación social que demuestre que no existe otra sociedad con el mismo nombre.
3. Estatutos sociales.
4. Acreditación del desembolso del capital social, ya sea en efectivo o mediante certificación bancaria.
5. En caso de realizar aportaciones no dinerarias, se requerirá un informe de experto sobre su valoración o un informe sustitutivo del administrador.
6. Relación detallada de los bienes aportados, incluyendo descripción, valoración y numeración de acciones o participaciones.

Además, al solicitar la inscripción en el Registro Mercantil, deberás presentar:
1. Escritura Pública de constitución de la Sociedad.
2. Fotocopia del Número de Identificación Fiscal (NIF).
3. Liquidación del Impuesto de Transmisiones Patrimoniales.

Recuerda que estos son los documentos básicos necesarios, y dependiendo

#### FUNCIÓN DE RESPUESTA

In [31]:
def get_answer(question):
    return chain.invoke(question)

#### INTERFAZ DE PRUEBA

In [32]:
iface = gr.Interface(fn=get_answer, inputs=gr.Textbox(
    value="Pregúntame lo que necesites"),
    live=False, 
    outputs="markdown",  
    title="Inteligencia sobre documentación mediante LLM, APIS y DBVectoriales",
    description="Puedes preguntar cualquier pregunta la creación de empresas en España",
    theme=gr.themes.Soft(),
    allow_flagging="never",)

iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


#### GUARDAMOS NUESTRA DB CHROMA EN UN ARCHIVO

In [37]:
#CHROMA SAVING IN SQLITE
persist_directory = "chromadb"
vectordb = Chroma.from_documents(documents=pages, embedding=embedding,
                                 persist_directory=persist_directory)
vectordb.persist()